In [2]:
from langchain.docstore.document import Document

In [4]:
import json
document_pages = json.load(open("/data/document_pages.json"))

In [9]:
md_pages = [Document(page_content=item['md']) for item in document_pages]

In [14]:
!pip install langchain-openai

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [5]:
from langchain.docstore.document import Document

import json
document_pages = json.load(open("/data/document_pages.json"))

md_pages = [Document(page_content=item['md']) for item in document_pages]

In [9]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(md_pages, testset_size=20)

Applying SummaryExtractor:  65%|██████▌   | 39/60 [00:08<00:04,  4.35it/s]Property 'summary' already exists in node '1bc243'. Skipping!
Property 'summary' already exists in node '4c6ac2'. Skipping!
Property 'summary' already exists in node '384eb3'. Skipping!
Applying SummaryExtractor:  73%|███████▎  | 44/60 [00:09<00:02,  5.89it/s]Property 'summary' already exists in node '7b4c1f'. Skipping!
Property 'summary' already exists in node 'c6f50a'. Skipping!
Applying SummaryExtractor:  75%|███████▌  | 45/60 [00:09<00:03,  4.98it/s]Property 'summary' already exists in node '1497f0'. Skipping!
Property 'summary' already exists in node 'd0f70e'. Skipping!
Property 'summary' already exists in node 'ae2ef3'. Skipping!
Applying SummaryExtractor:  80%|████████  | 48/60 [00:10<00:01,  6.63it/s]Property 'summary' already exists in node '4e9cfe'. Skipping!
Property 'summary' already exists in node '257cd2'. Skipping!
Property 'summary' already exists in node '91ccb6'. Skipping!
Applying SummaryExtrac

In [14]:
df = dataset.to_pandas()

In [16]:
import asyncio
from src.query import query
queries = df['user_input'].to_list()

res_buffer = []
context_buffer = []
for query_string in queries:
    print(query_string)
    print("=====================")
    res, context = asyncio.run(query(query_string))
    print(res)
    res_buffer.append(res)
    context_buffer.append(context)
    print("=====================")

df['answer'] = res_buffer
df['contexts'] = context_buffer

how particles move when perfume spill and why it spread in room?
When perfume is spilled, tiny particles called molecules evaporate and become a gas. These gas particles have energy and move around in all directions randomly. Because there are more perfume particles near the spill and fewer farther away, the particles spread out from where there are many to where there are fewer. This spreading out is called diffusion, and it makes the perfume smell spread throughout the room.
how copper(II) sulfate crystal do in water?
When a copper(II) sulfate crystal is dropped into water, it dissolves and the blue color spreads through the water. This happens because the copper(II) sulfate particles move from where there are many (the crystal) to where there are fewer (the water), a process called diffusion.
Why cells gotta be small in biology and why big animals got many small cells not one big cell, can you explain long?
Cells need to be small because they have to move oxygen, food, and waste mat

In [18]:
data_samples = {
    'question': df['user_input'].to_list(),
    'answer': df['answer'].to_list(),
    'contexts': [[item] for item in df['contexts'].to_list()],
    'ground_truth': df['reference'].to_list()
}
from datasets import Dataset
dataset = Dataset.from_dict(data_samples)

In [20]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating: 100%|██████████| 84/84 [01:46<00:00,  1.27s/it]


{'context_precision': 1.0000, 'faithfulness': 0.9135, 'answer_relevancy': 0.9166, 'context_recall': 0.9881}

In [21]:
df.to_csv("../data/evaluation_data.csv")